### Celda 1 – Imports y configuración básica

En esta celda:

- Importamos las librerías necesarias para:
  - Manejar datos (`pandas`, `numpy`)
  - Crear representaciones numéricas de texto (TF-IDF con `scikit-learn`)
  - Calcular similitud de coseno entre perfumes
  - Crear una pequeña interfaz interactiva en el propio Jupyter Notebook (`ipywidgets`)




In [2]:
# ============================================
# Celda 1: Imports y configuración básica
# ============================================

# Si necesitas instalar algo, descomenta:
# !pip install pandas scikit-learn ipywidgets

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from ipywidgets import widgets, VBox, Layout
from IPython.display import display


### Celda 2 – Carga del CSV `fra_cleaned.csv` y vista rápida

En esta celda:

- Cargamos el archivo `fra_cleaned.csv` que has adjuntado.
- Este CSV ya está bastante limpio, pero aun así haremos una limpieza extra en la siguiente celda.
- Mostramos algunas filas para ver la estructura y confirmamos las columnas disponibles.




In [3]:
# ============================================
# Celda 2: Carga del dataset y vista rápida
# ============================================

# Nombre del archivo que has adjuntado
ruta_csv = "fra_cleaned.csv"

# Este CSV viene con separador ';' y codificación latina (acentos, etc.)
df_raw = pd.read_csv(ruta_csv, sep=';', encoding='latin-1')

print("Dimensiones del dataset original (filas, columnas):", df_raw.shape)
display(df_raw.head())

print("\nColumnas disponibles en el dataset:")
print(df_raw.columns.tolist())


Dimensiones del dataset original (filas, columnas): (24063, 18)


,url,Perfume,Brand,Country,Gender,Rating Value,Rating Count,Year,Top,Middle,Base,Perfumer1,Perfumer2,mainaccord1,mainaccord2,mainaccord3,mainaccord4,mainaccord5
0,https://www.fragrantica.com/perfume/xerjoff/ac...,accento-overdose-pride-edition,xerjoff,Italy,unisex,"1,42",201,2022.0,"fruity notes, aldehydes, green notes","bulgarian rose, egyptian jasmine, lily-of-the-...","eucalyptus, pine",unknown,NaN,rose,woody,fruity,aromatic,floral
1,https://www.fragrantica.com/perfume/jean-paul-...,classique-pride-2024,jean-paul-gaultier,France,women,"1,86",70,2024.0,"yuzu, citruses","orange blossom, neroli","musk, blonde woods",unknown,NaN,citrus,white floral,sweet,fresh,musky
2,https://www.fragrantica.com/perfume/jean-paul-...,classique-pride-2023,jean-paul-gaultier,France,unisex,"1,91",285,2023.0,"blood orange, yuzu","neroli, orange blossom","musk, white woods",natalie gracia-cetto,quentin bisch,citrus,white floral,sweet,fresh spicy,musky
3,https://www.fragrantica.com/perfume/bruno-bana...,pride-edition-man,bruno-banani,Germany,men,"1,92",59,2019.0,"guarana, grapefruit, red apple","walnut, lavender, guava","vetiver, benzoin, amber",unknown,NaN,fruity,nutty,woody,tropical,NaN
4,https://www.fragrantica.com/perfume/jean-paul-...,le-male-pride-collector,jean-paul-gaultier,France,men,"1,93",632,2020.0,"mint, lavender, cardamom, artemisia, bergamot","caraway, cinnamon, orange blossom","vanilla, sandalwood, amber, cedar, tonka bean",francis kurkdjian,NaN,aromatic,warm spicy,fresh spicy,cinnamon,vanilla



Columnas disponibles en el dataset:
['url', 'Perfume', 'Brand', 'Country', 'Gender', 'Rating Value', 'Rating Count', 'Year', 'Top', 'Middle', 'Base', 'Perfumer1', 'Perfumer2', 'mainaccord1', 'mainaccord2', 'mainaccord3', 'mainaccord4', 'mainaccord5']


### Celda 3 – Limpieza básica de datos

Aunque `fra_cleaned.csv` ya viene bastante “limpio”, aquí hacemos una limpieza adicional:

1. Creamos una copia de trabajo (`df`) a partir de `df_raw`.
2. Eliminamos filas duplicadas.
3. Comprobamos que la columna `Perfume` (nombre de la colonia) no esté vacía.
4. Quitamos espacios en blanco al principio y al final de todas las cadenas de texto.
5. Convertimos:
   - `Rating Value` en número (sustituyendo coma decimal por punto).
   - `Rating Count` en entero, si procede.
   - `Year` en número (por si luego quisieras usarlo).

El objetivo es dejar el DataFrame listo para el modelo de recomendación.


In [4]:
# ============================================
# Celda 3: Limpieza básica de datos (mejorada)
# ============================================

df = df_raw.copy()

print("Dimensiones antes de limpiar:", df.shape)

# 1) Eliminar duplicados
df.drop_duplicates(inplace=True)

# 2) Asegurarnos de que 'Perfume' tiene algo de texto
df["Perfume"] = df["Perfume"].astype(str)
df = df[df["Perfume"].str.strip() != ""]

# 3) Limpiar cadenas de texto:
#    - strip espacios al inicio/final
#    - reemplazar GUIONES por ESPACIOS
#    - colapsar dobles espacios
for col in df.columns:
    if df[col].dtype == "object":
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()
            .str.replace("-", " ")                 # 👈 guiones → espacios
            .str.replace(r"\s+", " ", regex=True)  # 👈 colapsar múltiples espacios
        )

# 4) Convertir columnas numéricas (rating y año)

def convertir_rating_valor(x):
    if pd.isna(x):
        return np.nan
    s = str(x).replace(',', '.')   # por si viene '4,5'
    try:
        return float(s)
    except ValueError:
        return np.nan

df["Rating Value"] = df["Rating Value"].apply(convertir_rating_valor)

def convertir_rating_count(x):
    if pd.isna(x):
        return np.nan
    try:
        return int(x)
    except ValueError:
        return np.nan

df["Rating Count"] = df["Rating Count"].apply(convertir_rating_count)

def convertir_year(x):
    if pd.isna(x):
        return np.nan
    try:
        return float(x)
    except ValueError:
        return np.nan

df["Year"] = df["Year"].apply(convertir_year)

print("Dimensiones después de limpiar:", df.shape)
display(df.head())


Dimensiones antes de limpiar: (24063, 18)
Dimensiones después de limpiar: (24063, 18)


,url,Perfume,Brand,Country,Gender,Rating Value,Rating Count,Year,Top,Middle,Base,Perfumer1,Perfumer2,mainaccord1,mainaccord2,mainaccord3,mainaccord4,mainaccord5
0,https://www.fragrantica.com/perfume/xerjoff/ac...,accento overdose pride edition,xerjoff,Italy,unisex,1.42,201,2022.0,"fruity notes, aldehydes, green notes","bulgarian rose, egyptian jasmine, lily of the ...","eucalyptus, pine",unknown,nan,rose,woody,fruity,aromatic,floral
1,https://www.fragrantica.com/perfume/jean paul ...,classique pride 2024,jean paul gaultier,France,women,1.86,70,2024.0,"yuzu, citruses","orange blossom, neroli","musk, blonde woods",unknown,nan,citrus,white floral,sweet,fresh,musky
2,https://www.fragrantica.com/perfume/jean paul ...,classique pride 2023,jean paul gaultier,France,unisex,1.91,285,2023.0,"blood orange, yuzu","neroli, orange blossom","musk, white woods",natalie gracia cetto,quentin bisch,citrus,white floral,sweet,fresh spicy,musky
3,https://www.fragrantica.com/perfume/bruno bana...,pride edition man,bruno banani,Germany,men,1.92,59,2019.0,"guarana, grapefruit, red apple","walnut, lavender, guava","vetiver, benzoin, amber",unknown,nan,fruity,nutty,woody,tropical,nan
4,https://www.fragrantica.com/perfume/jean paul ...,le male pride collector,jean paul gaultier,France,men,1.93,632,2020.0,"mint, lavender, cardamom, artemisia, bergamot","caraway, cinnamon, orange blossom","vanilla, sandalwood, amber, cedar, tonka bean",francis kurkdjian,nan,aromatic,warm spicy,fresh spicy,cinnamon,vanilla


### Celda 4 – Preparar `df_perfumes` y crear `ingredientes_texto`

En esta celda:

1. Seleccionamos solo las columnas relevantes para el recomendador:
   - `Perfume` → `name`
   - `Brand` → `brand`
   - `Gender` → `gender_raw`
   - `Top`, `Middle`, `Base` → notas
   - `mainaccord1` … `mainaccord5` → acordes principales

2. Normalizamos el género (`Gender`) en una nueva columna `gender`:
   - `men` → `male`
   - `women` → `female`
   - `unisex` → `unisex`

3. Generamos la columna `ingredientes_texto`:
   - concatenamos el contenido de `Top`, `Middle`, `Base`, `mainaccord1`…`mainaccord5`
   - convertimos todo a minúsculas
   - esta columna es la que usaremos para calcular similitudes entre perfumes.

Finalmente quitamos filas que no tengan ningún texto olfativo.


In [5]:
# ============================================
# Celda 4: Preparar df_perfumes y 'ingredientes_texto'
# ============================================

# Creamos un DataFrame con solo las columnas necesarias para el recomendador
columnas_usar = [
    "Perfume", "Brand", "Gender",
    "Top", "Middle", "Base",
    "mainaccord1", "mainaccord2", "mainaccord3", "mainaccord4", "mainaccord5"
]

# Nos aseguramos de que existen (por si acaso)
columnas_existentes = [c for c in columnas_usar if c in df.columns]
df_perfumes = df[columnas_existentes].copy()

# Renombramos a nombres internos más cómodos
df_perfumes.rename(columns={
    "Perfume": "name",
    "Brand": "brand",
    "Gender": "gender_raw"
}, inplace=True)

# Normalizamos el género
def normalizar_genero(g):
    if pd.isna(g):
        return None

    s = str(g).strip().lower()

    # Primero detectar femenino para evitar que "women" coincida con "men"
    if "women" in s or "woman" in s:
        return "female"

    # Luego masculino
    if "men" in s or "man" in s:
        return "male"

    if "unisex" in s:
        return "unisex"

    return s


df_perfumes["gender"] = df_perfumes["gender_raw"].apply(normalizar_genero)

# Rellenamos NaN con cadenas vacías en las columnas de notas/acordes
for col in ["Top", "Middle", "Base",
            "mainaccord1", "mainaccord2", "mainaccord3", "mainaccord4", "mainaccord5"]:
    if col in df_perfumes.columns:
        df_perfumes[col] = df_perfumes[col].fillna("")

# Creamos la columna 'ingredientes_texto'
def crear_ingredientes_texto(row):
    partes = []
    for col in ["Top", "Middle", "Base",
                "mainaccord1", "mainaccord2", "mainaccord3", "mainaccord4", "mainaccord5"]:
        if col in row and isinstance(row[col], str):
            partes.append(row[col])
    return " ".join(partes).lower()

df_perfumes["ingredientes_texto"] = df_perfumes.apply(crear_ingredientes_texto, axis=1)

# Eliminamos perfumes sin información olfativa
df_perfumes = df_perfumes[df_perfumes["ingredientes_texto"].str.strip() != ""]

print("Dimensiones finales de df_perfumes:", df_perfumes.shape)
df_perfumes.head()


Dimensiones finales de df_perfumes: (24063, 13)


,name,brand,gender_raw,Top,Middle,Base,mainaccord1,mainaccord2,mainaccord3,mainaccord4,mainaccord5,gender,ingredientes_texto
0,accento overdose pride edition,xerjoff,unisex,"fruity notes, aldehydes, green notes","bulgarian rose, egyptian jasmine, lily of the ...","eucalyptus, pine",rose,woody,fruity,aromatic,floral,unisex,"fruity notes, aldehydes, green notes bulgarian..."
1,classique pride 2024,jean paul gaultier,women,"yuzu, citruses","orange blossom, neroli","musk, blonde woods",citrus,white floral,sweet,fresh,musky,female,"yuzu, citruses orange blossom, neroli musk, bl..."
2,classique pride 2023,jean paul gaultier,unisex,"blood orange, yuzu","neroli, orange blossom","musk, white woods",citrus,white floral,sweet,fresh spicy,musky,unisex,"blood orange, yuzu neroli, orange blossom musk..."
3,pride edition man,bruno banani,men,"guarana, grapefruit, red apple","walnut, lavender, guava","vetiver, benzoin, amber",fruity,nutty,woody,tropical,nan,male,"guarana, grapefruit, red apple walnut, lavende..."
4,le male pride collector,jean paul gaultier,men,"mint, lavender, cardamom, artemisia, bergamot","caraway, cinnamon, orange blossom","vanilla, sandalwood, amber, cedar, tonka bean",aromatic,warm spicy,fresh spicy,cinnamon,vanilla,male,"mint, lavender, cardamom, artemisia, bergamot ..."


### Celda 5 – Vectorización TF-IDF de las notas/ingredientes

En esta celda:

- Convertimos la columna `ingredientes_texto` en vectores numéricos usando **TF-IDF**.
- Cada perfume se convierte en un vector en un espacio de características (una por palabra/nota).
- Guardamos:
  - `vectorizer`: el objeto TF-IDF entrenado.
  - `ingredientes_matrix`: la matriz de vectores para todos los perfumes.


In [6]:
# ============================================
# Celda 5: Vectorización TF-IDF
# ============================================

vectorizer = TfidfVectorizer()

ingredientes_matrix = vectorizer.fit_transform(df_perfumes["ingredientes_texto"])

ingredientes_matrix.shape


(24063, 1274)

In [7]:
# ============================================
# Celda X: Similitud por notas y acordes
# PRIORIDAD: acordes > fondo > corazón > salida
# Usa columnas: Top, Middle, Base, mainaccord1..5
# ============================================

from typing import Set
import math

def normalizar_nota(note: str) -> str:
    """
    Normaliza una nota:
    - la convierte a string
    - minúsculas
    - sin espacios sobrantes
    """
    return str(note).strip().lower()


def valor_a_set_notas(valor) -> Set[str]:
    """
    Convierte el contenido de una columna de notas en un set normalizado.
    Acepta:
    - listas/tuplas/sets de strings
    - strings con notas separadas por coma o punto y coma
    - NaN / None -> set()
    """
    if valor is None:
        return set()

    # Evitar problemas con NaN de pandas
    try:
        if isinstance(valor, float) and math.isnan(valor):
            return set()
    except Exception:
        pass

    # Si ya es lista / tupla / set
    if isinstance(valor, (list, tuple, set)):
        return {normalizar_nota(v) for v in valor if v and str(v).strip()}

    # Si es string: separar por "," o ";"
    if isinstance(valor, str):
        bruto = valor.replace(";", ",")
        partes = [p.strip() for p in bruto.split(",")]
        return {normalizar_nota(p) for p in partes if p}

    # Cualquier otro tipo -> string simple
    return {normalizar_nota(valor)}


def jaccard_similarity(a: Set[str], b: Set[str]) -> float:
    """
    Similitud de Jaccard entre dos conjuntos (0–1).
    """
    if not a and not b:
        return 0.0
    inter = len(a & b)
    union = len(a | b)
    return inter / union if union > 0 else 0.0


def calcular_similitud_row(
    row_ref,
    row_cand,
    # columnas reales de tu DataFrame
    cols_notas = ("Top", "Middle", "Base"),
    cols_accords = ("mainaccord1", "mainaccord2", "mainaccord3", "mainaccord4", "mainaccord5"),
    # pesos: PRIORIDAD -> acordes > fondo > corazón > salida
    weight_accords: float = 0.4,
    weight_base: float = 0.3,
    weight_heart: float = 0.2,
    weight_top: float = 0.1,
) -> float:
    """
    Score global (0–1) entre dos filas del DataFrame.

    - Acordes principales (mainaccord1..5)  -> peso_accords
    - Notas de fondo (Base)                -> peso_base
    - Notas de corazón (Middle)            -> peso_heart
    - Notas de salida (Top)                -> peso_top

    Cuanto más alto, más parecido.
    """

    col_top, col_heart, col_base = cols_notas

    # --- sets de notas ---
    ref_top = valor_a_set_notas(row_ref.get(col_top)) if col_top in row_ref else set()
    ref_heart = valor_a_set_notas(row_ref.get(col_heart)) if col_heart in row_ref else set()
    ref_base = valor_a_set_notas(row_ref.get(col_base)) if col_base in row_ref else set()

    cand_top = valor_a_set_notas(row_cand.get(col_top)) if col_top in row_cand else set()
    cand_heart = valor_a_set_notas(row_cand.get(col_heart)) if col_heart in row_cand else set()
    cand_base = valor_a_set_notas(row_cand.get(col_base)) if col_base in row_cand else set()

    s_top = jaccard_similarity(ref_top, cand_top)
    s_heart = jaccard_similarity(ref_heart, cand_heart)
    s_base = jaccard_similarity(ref_base, cand_base)

    # --- acordes principales ---
    ref_acc = set()
    cand_acc = set()
    for c in cols_accords:
        if c in row_ref:
            ref_acc |= valor_a_set_notas(row_ref[c])
        if c in row_cand:
            cand_acc |= valor_a_set_notas(row_cand[c])

    s_accords = jaccard_similarity(ref_acc, cand_acc) if (ref_acc or cand_acc) else 0.0

    # --- combinar con pesos (acordes > base > corazón > top) ---
    num = (
        weight_accords * s_accords +
        weight_base * s_base +
        weight_heart * s_heart +
        weight_top * s_top
    )
    den = weight_accords + weight_base + weight_heart + weight_top

    if den <= 0:
        return 0.0

    score = num / den
    # asegurar rango 0–1
    score = max(0.0, min(1.0, float(score)))
    return score


### Celda 6 – Funciones de búsqueda y recomendación

En esta celda definimos dos funciones clave:

1. `buscar_indice_por_nombre(nombre_perfume, df_local)`  
   - Dado un nombre (o parte del nombre) de un perfume:
     - Busca coincidencias en la columna `name` sin distinguir mayúsculas.
     - Devuelve el índice de la primera coincidencia o `None` si no encuentra nada.

2. `recomendar_perfumes(nombre_perfume=None, ingredientes_gustados=None, genero=None, n_resultados=5)`  
   - Parámetros:
     - `nombre_perfume`: nombre de una colonia que le gusta al usuario (opcional).
     - `ingredientes_gustados`: ingredientes que le gustan (string con comas o lista) (opcional).
     - `genero`: filtro de género (`'male'`, `'female'`, `'unisex'`), opcional.
     - `n_resultados`: número de colonias a devolver (por defecto 5).
   - Lógica:
     1. Filtrar por género si se ha proporcionado.
     2. Construir un **vector base**:
        - el vector del perfume favorito (si lo hay)
        - combinado con el vector de los ingredientes favoritos (si los hay)
     3. Calcular la similitud de coseno entre ese vector base y todos los perfumes candidatos.
     4. Devolver los `n_resultados` perfumes más similares.

   - Si el usuario no introduce ni colonia favorita ni ingredientes, devolvemos simplemente algunas colonias de ejemplo del conjunto filtrado.


In [8]:
# ============================================
# Celda 6: Búsqueda por nombre+marca y recomendador
# Similitud basada en notas y acordes:
# PRIORIDAD = acordes > fondo > corazón > salida
# ============================================

def normalizar_texto(s):
    """
    Normaliza texto para comparación robusta:
    - minúsculas
    - quita espacios extremos
    - convierte guiones en espacios
    - colapsa espacios múltiples
    """
    s = str(s).lower().strip()
    s = s.replace("-", " ")
    s = " ".join(s.split())
    return s


def buscar_indice_por_nombre_y_marca(nombre_perfume, marca, df_local):
    """
    Busca un perfume usando nombre + marca, tolerante con mayúsculas,
    tildes y espacios.
    """
    nombre_norm = normalizar_texto(nombre_perfume) if nombre_perfume else ""
    marca_norm = normalizar_texto(marca) if marca else ""

    if not nombre_norm:
        return None

    df_work = df_local.copy()
    df_work["_name_norm"]  = df_work["name"].astype(str).apply(normalizar_texto)
    df_work["_brand_norm"] = df_work["brand"].astype(str).apply(normalizar_texto)

    # 1) Si hay marca → exacto
    if marca_norm:
        exactos = df_work[
            (df_work["_name_norm"] == nombre_norm) &
            (df_work["_brand_norm"] == marca_norm)
        ]
        if len(exactos) > 0:
            return exactos.index[0]

        # 2) parcial
        parciales = df_work[
            df_work["_name_norm"].str.contains(nombre_norm, na=False) &
            df_work["_brand_norm"].str.contains(marca_norm, na=False)
        ]
        if len(parciales) > 0:
            return parciales.index[0]

    # 3) Sin marca → buscar solo por nombre
    parciales = df_work[df_work["_name_norm"].str.contains(nombre_norm, na=False)]
    if len(parciales) > 0:
        return parciales.index[0]

    return None


# ---------------------------------------------
# FILTRO DE ACORDES PRINCIPALES (mainaccord1..5)
# ---------------------------------------------
def _filtrar_por_acordes(df_in, texto_usuario):
    """
    Filtra df_in para que aparezcan TODOS los acordes que el usuario escribe
    en alguna de las columnas mainaccord1..5.
    """
    texto_usuario = (texto_usuario or "").strip()
    if not texto_usuario:
        return df_in

    acordes = [
        a.strip().lower()
        for a in texto_usuario.replace(";", ",").split(",")
        if a.strip()
    ]
    if not acordes:
        return df_in

    cols_accords = ["mainaccord1", "mainaccord2", "mainaccord3", "mainaccord4", "mainaccord5"]

    df_filtrado = df_in.copy()

    for acorde in acordes:
        mascara = False
        for col in cols_accords:
            if col in df_filtrado.columns:
                mascara = mascara | df_filtrado[col].astype(str).str.lower().str.contains(acorde, na=False)

        df_filtrado = df_filtrado[mascara]

        if len(df_filtrado) == 0:
            return df_filtrado

    return df_filtrado


# ---------------------------------------------
# RECOMENDADOR PRINCIPAL
# ---------------------------------------------
def recomendar_perfumes(
    nombre_perfume: str,
    marca: str = "",
    genero: str = "",
    acordes_principales: str = "",
    n_resultados: int = 20,
):
    """
    Recomienda perfumes basados en:
    1) Acordes principales (mainaccord1..5)
    2) Notas de fondo (Base)
    3) Notas de corazón (Middle)
    4) Notas de salida (Top)

    Filtra después por acordes que indique el usuario.
    """

    # --- 1) Buscar perfume de referencia ---
    idx_ref = buscar_indice_por_nombre_y_marca(nombre_perfume, marca, df_perfumes)
    if idx_ref is None:
        print("No se ha encontrado ese perfume en la base de datos.")
        return pd.DataFrame()

    row_ref = df_perfumes.loc[idx_ref]

    # --- 2) Construir candidatos ---
    candidatos = df_perfumes.copy()
    candidatos = candidatos[candidatos.index != idx_ref]

    # --- 3) Filtro de género (opcional) ---
    genero = (genero or "").strip().lower()
    if genero and genero not in ["", "all", "todos", "cualquiera"]:
        if "gender" in candidatos.columns:
            candidatos = candidatos[
                candidatos["gender"].astype(str).str.lower() == genero
            ]

    if len(candidatos) == 0:
        print("No hay perfumes que coincidan con ese género.")
        return pd.DataFrame()

    # --- 4) Calcular similitud ---
    similitudes = []
    for idx, row in candidatos.iterrows():
        sim = calcular_similitud_row(
            row_ref=row_ref,
            row_cand=row
            # Usa los pesos definidos en Celda X:
            # acordes > fondo > corazón > salida
        )
        similitudes.append(sim)

    candidatos = candidatos.copy()
    candidatos["similarity"] = similitudes

    candidatos_ordenados = candidatos.sort_values("similarity", ascending=False)

    # --- 5) Filtro final por acordes que el usuario pide ---
    candidatos_filtrados = _filtrar_por_acordes(candidatos_ordenados, acordes_principales)

    if len(candidatos_filtrados) == 0:
        print("Se encontró la colonia de referencia, pero ningún perfume tiene esos acordes.")
        return pd.DataFrame()

    return candidatos_filtrados.head(n_resultados)


### Celda 7 – Interfaz tipo formulario con ipywidgets

En esta celda creamos una pequeña interfaz gráfica dentro del Jupyter Notebook:

Campos del formulario:

- **Colonia favorita** (texto, opcional)  
- **Género** (desplegable, opcional: `male`, `female`, `unisex`)  
- **Ingredientes que le gustan** (texto, opcional, separados por comas)  
  - Ejemplo: `vainilla, ámbar, rosa`

Funcionamiento:

- Si solo indicas colonia favorita → colonias con notas similares.
- Si solo indicas ingredientes → colonias similares a ese perfil olfativo.
- Si indicas ambos → combinación de los dos criterios.
- Si además seleccionas género → se aplica primero el filtro de género.

No hay filtro por precio porque este CSV no contiene precios.


In [9]:

# ============================================
# Celda 7: Interfaz con ipywidgets (marca + nombre + acordes)
# ============================================

# Campo de texto para la marca
marca_widget = widgets.Text(
    description="Marca:",
    placeholder="Ej: Dior",
    layout=Layout(width="300px")
)

# Campo de texto para la colonia favorita
nombre_widget = widgets.Text(
    description="Colonia:",
    placeholder="Ej: Sauvage",
    layout=Layout(width="400px")
)

# Dropdown de género (opcional)
opciones_genero = ["", "male", "female", "unisex"]

genero_widget = widgets.Dropdown(
    options=opciones_genero,
    value="",
    description="Género:"
)

# Campo para acordes principales
acordes_widget = widgets.Text(
    description="Acordes:",
    placeholder="Ej: woody, amber, citrus",
    layout=Layout(width="500px")
)

# Botón para lanzar la búsqueda
boton_buscar = widgets.Button(
    description="Buscar colonias",
    button_style="success"
)

# Área de salida para resultados
salida = widgets.Output()

def on_boton_click(b):
    salida.clear_output()
    with salida:
        marca = marca_widget.value.strip()
        nombre = nombre_widget.value.strip()
        genero = genero_widget.value.strip() or ""
        acordes_texto = acordes_widget.value.strip()

        recomendaciones = recomendar_perfumes(
            nombre_perfume=nombre if nombre else None,
            marca=marca if marca else "",
            genero=genero,
            acordes_principales=acordes_texto,
            n_resultados=20
        )

        if recomendaciones is None or len(recomendaciones) == 0:
            print("No se han encontrado colonias con los criterios indicados.")
        else:
            # Mostramos columnas relevantes
            columnas_mostrar = [
                "brand", "name", "gender", "similarity",
                "Top", "Middle", "Base",
                "mainaccord1", "mainaccord2", "mainaccord3", "mainaccord4", "mainaccord5"
            ]
            columnas_mostrar = [c for c in columnas_mostrar if c in recomendaciones.columns]
            display(recomendaciones[columnas_mostrar])

boton_buscar.on_click(on_boton_click)

formulario = VBox([
    marca_widget,
    nombre_widget,
    genero_widget,
    acordes_widget,
    boton_buscar,
    salida
])

display(formulario)
